In [ ]:
from SPIDER import train_SPIDER, Cal_knn_expression, Cal_Spatial_Net, mclust_R
import os, pickle, pandas as pd
import scanpy as sc, numpy as np
import  sklearn
from sklearn.metrics import normalized_mutual_info_score, homogeneity_score
from sklearn.metrics.cluster import adjusted_rand_score

Download the DLPFC processed data from this [link](https://drive.google.com/file/d/14sieoleV-a8Hx9KVVWDFZk5-5MjGsRLT/view?usp=sharing). Put it inside the data folder. 

In [ ]:
## Data includes preprocessed real ST and pseudo ST geenrated from reference scRNA-seq
with open("data/DLPFC_data.pickle", "rb") as f:
    loaded = pickle.load(f)
# Change the code accordingly to use the CID44971 data
std, psd = loaded["realST"], loaded["pseudoST"]

In [ ]:
params = {'W_recon': 3.5611961149150226, 'W_mmd': 1.109798809633746, 'W_cell': 1.280970162059623, 'k_cutoff': 9, 'lr': 0.0004790792935102623}
k_cutoff = params["k_cutoff"]
rad_cutoff = std.uns['rad_cutoff']
Cal_knn_expression(std, k_cutoff=k_cutoff)
Cal_knn_expression(psd, k_cutoff=k_cutoff)
Cal_Spatial_Net(std, rad_cutoff=rad_cutoff)

In [ ]:
std,model = train_SPIDER(std, psd, key_added='SPIDER',lr=params["lr"],n_epochs=500,device='cuda:0',random_seed=3156,
                                W_recon=params["W_recon"],W_mmd=params["W_mmd"],W_cell=params["W_cell"])

In [ ]:
sc.pp.normalize_total(std,layer='SPIDER',target_sum=1e4)
sc.pp.log1p(std,layer='SPIDER')
std.obsm['SPIDER_pca'] = sc.tl.pca(std.layers['SPIDER'], n_comps=20, svd_solver="arpack", random_state=3156)
mclust_R(std, used_obsm='SPIDER_pca',num_cluster=std.uns['num_cluster'],modelNames="EEI",random_seed=3156)
sc.pl.embedding(std,basis="spatial", color=["mclust","Ground Truth"],s=100, title=["SPIDER","Ground Truth"])

In [ ]:
ARI = adjusted_rand_score(std.obs['mclust'], std.obs['Ground Truth'])
NMI = normalized_mutual_info_score(std.obs['mclust'], std.obs['Ground Truth'])
HS = homogeneity_score(std.obs['mclust'], std.obs['Ground Truth'])
print(ARI, NMI, HS)